In [3]:
import psycopg2
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as plot
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots


In [4]:
conn = psycopg2.connect(
           dbname="qdap_test",
           user="amt",
           password="your_password",
           host="192.168.2.23",
           port="5432"
       )

# Create a cursor object using the cursor() method
cursor = conn.cursor()

def fetch_options(cursor, symbol, expiry):
    query = '''
        SELECT * 
        FROM ohlcv_options_per_minute oopm
        WHERE symbol = %s
        AND expiry_type = 'I'
        AND expiry = %s
        ORDER BY date_timestamp;
        '''
    cursor.execute(query,(symbol,expiry))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df
# Create a cursor object
cursor = conn.cursor()

def fetch_futures(cursor, symbol, x):
    query = '''
        SELECT *
        FROM ohlcv_future_per_minute ofpm 
        WHERE ofpm.symbol = %s
        AND ofpm.expiry_type = 'I'
        AND DATE(ofpm.expiry) = %s
        ORDER BY date_timestamp ASC
    '''
    # Execute the query with parameters as a tuple
    cursor.execute(query, (symbol, x))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

# Define the fetch_expiries function
def fetch_expiries(cursor, symbol):
    query = f'''
        SELECT DISTINCT ofpem.expiry 
        FROM ohlcv_future_per_minute ofpem 
        WHERE ofpem.symbol = '{symbol}'
        AND ofpem.expiry_type = 'I'
        GROUP BY ofpem.expiry 
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

In [5]:
symbol = 'BANKNIFTY'  # Replace with your symbol
expiry_future = fetch_expiries(cursor, symbol)

# Convert the 'expiry' column to datetime and then to date
expiry_future['expiry'] = pd.to_datetime(expiry_future['expiry']).dt.date

# Print the expiry dates with their indices
for i, row in expiry_future.iterrows():
    print(i, row.iloc[0], sep=' = ', end=' | ')
    if (i + 1) % 6 == 0:
        print()

0 = 2022-05-26 | 1 = 2022-06-30 | 2 = 2022-07-28 | 3 = 2022-08-25 | 4 = 2022-09-29 | 5 = 2022-10-27 | 
6 = 2022-11-24 | 7 = 2022-12-29 | 8 = 2023-01-25 | 9 = 2023-02-23 | 10 = 2023-03-29 | 11 = 2023-04-27 | 
12 = 2023-05-25 | 13 = 2023-06-28 | 14 = 2023-06-29 | 15 = 2023-07-27 | 16 = 2023-08-31 | 17 = 2023-09-28 | 
18 = 2023-10-26 | 19 = 2023-11-30 | 20 = 2023-12-28 | 21 = 2024-01-25 | 22 = 2024-02-29 | 23 = 2024-03-27 | 
24 = 2024-04-24 | 25 = 2024-05-29 | 26 = 2024-06-26 | 27 = 2024-07-31 | 

In [11]:
index = 20

expiry_date = expiry_future.iloc[index]['expiry']
# Fetch the futures data for the selected expiry date
futures = fetch_futures(cursor, symbol, expiry_date)
# Convert the 'date_timestamp' column to datetime if it exists
futures = futures.drop(columns=['id'])
futures = futures.drop_duplicates()


In [14]:
def fill_missings(futures):
    # Generate the full range of timestamps for each day
    start_time = '09:15:00'
    end_time = '15:30:00'

    all_timestamps = pd.date_range(start=start_time, end=end_time, freq='min').time

    # Generate a DataFrame with all timestamps for each day in the data
    all_dates = futures['date_timestamp'].dt.date.unique()
    all_date_times = [pd.Timestamp.combine(date, time) for date in all_dates for time in all_timestamps]
    all_date_times_df = pd.DataFrame(all_date_times, columns=['date_timestamp'])


    # Merge with the original DataFrame to include all timestamps
    futures_full = pd.merge(all_date_times_df, futures, on='date_timestamp', how='left')

    # Forward and backword fill missing values
    futures_full = futures_full.ffill().bfill()

    # Sort by date_timestamp to maintain order
    futures_full = futures_full.sort_values('date_timestamp').reset_index(drop=True)



In [15]:
def genrate_signals(futures , long_window = 26, short_window = 9):
    futures['Short_EMA'] = futures['close'].ewm(span=short_window, adjust=False).mean()
    futures['Long_EMA'] = futures['close'].ewm(span=long_window, adjust=False).mean()

    futures['Signal'] = 0 

    for i in range(1, len(futures)):
        if futures['Short_EMA'].iloc[i] > futures['Long_EMA'].iloc[i] and futures['Short_EMA'].iloc[i-1] <= futures['Long_EMA'].iloc[i-1]:
            futures.at[futures.index[i], 'Signal'] = 1  # Buy signal
        elif futures['Short_EMA'].iloc[i] < futures['Long_EMA'].iloc[i] and futures['Short_EMA'].iloc[i-1] >= futures['Long_EMA'].iloc[i-1]:
            futures.at[futures.index[i], 'Signal'] = -1  # Sell signal

In [16]:
def plot_signals(futures):
    
    fig = make_subplots(rows=1, cols=1)

    # Candlestick chart
    candlestick = go.Candlestick(
        x=futures.index,
        open=futures['open'],
        high=futures['high'],
        low=futures['low'],
        close=futures['close'],
        name='Candlesticks'
    )
    fig.add_trace(candlestick)

    # Short EMA
    short_ema = go.Scatter(x=futures.index, y=futures['Short_EMA'], mode='lines', name='Short EMA')
    fig.add_trace(short_ema)

    # Long EMA
    long_ema = go.Scatter(x=futures.index, y=futures['Long_EMA'], mode='lines', name='Long EMA')
    fig.add_trace(long_ema)

    # Buy signals
    buy_signals = go.Scatter(
        x=futures[futures['Signal'] == 1].index,
        y=futures['Short_EMA'][futures['Signal'] == 1],
        mode='markers',
        marker=dict(symbol='triangle-up', color='yellow', size=12),
        name='Buy Signal'
    )
    fig.add_trace(buy_signals)

    # Sell signals
    sell_signals = go.Scatter(
        x=futures[futures['Signal'] == -1].index,
        y=futures['Short_EMA'][futures['Signal'] == -1],
        mode='markers',
        marker=dict(symbol='triangle-down', color='black', size=12),
        name='Sell Signal'
    )
    fig.add_trace(sell_signals)

    fig.update_layout(
        title=f'{symbol} EMA Crossover Strategy',
        yaxis_title='Price',
        xaxis_title='Date',
        xaxis_rangeslider_visible=False,
        width=2000,
        height=1000
    )

    return fig

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# Assuming 'futures' is already defined as your DataFrame
# Ensure 'date_timestamp' is in datetime format
futures['date_timestamp'] = pd.to_datetime(futures['date_timestamp'])

fig = make_subplots(rows=1, cols=1)

# Candlestick chart
candlestick = go.Candlestick(
    x=futures['date_timestamp'],
    open=futures['open'],
    high=futures['high'],
    low=futures['low'],
    close=futures['close'],
    name='Candlesticks'
)
fig.add_trace(candlestick)

# Short EMA
short_ema = go.Scatter(x=futures['date_timestamp'], y=futures['Short_EMA'], mode='lines', name='Short EMA')
fig.add_trace(short_ema)

# Long EMA
long_ema = go.Scatter(x=futures['date_timestamp'], y=futures['Long_EMA'], mode='lines', name='Long EMA')
fig.add_trace(long_ema)

# Buy signals
buy_signals = go.Scatter(
    x=futures[futures['Signal'] == 1]['date_timestamp'],
    y=futures['Short_EMA'][futures['Signal'] == 1],
    mode='markers',
    marker=dict(symbol='triangle-up', color='yellow', size=12),
    name='Buy Signal'
)
fig.add_trace(buy_signals)

# Sell signals
sell_signals = go.Scatter(
    x=futures[futures['Signal'] == -1]['date_timestamp'],
    y=futures['Short_EMA'][futures['Signal'] == -1],
    mode='markers',
    marker=dict(symbol='triangle-down', color='black', size=12),
    name='Sell Signal'
)
fig.add_trace(sell_signals)

fig.update_layout(
    title=f'{symbol} EMA Crossover Strategy',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    width=2000,
    height=1000
)

pyo.plot(fig)

In [159]:

options = fetch_options(cursor, symbol=symbol, expiry=str(expiry_date))



314782 431655
912000 431655


In [ ]:
def fetch_options_data(options):

    options.drop(columns=['id'],inplace = True)
    options.drop_duplicates(inplace=True)

    options['date_timestamp'] = pd.to_datetime(options['date_timestamp'])
    copt = options[options['opt_type']=='CE'].copy()
    popt = options[options['opt_type']=='PE'].copy()

    copt.drop_duplicates(inplace = True)

    popt.drop_duplicates(inplace = True)

    strike_values = options['strike'].unique()
    all_timestamps = options['date_timestamp'].unique()

    d2c = [[j, i]  for j in all_timestamps for i in strike_values]

    d2c_df = pd.DataFrame(d2c , columns = ['date_timestamp' , 'strike'])

    ce = pd.merge(d2c_df , copt,on=['date_timestamp','strike'] , how='left')
    pe = pd.merge(d2c_df , popt,on=['date_timestamp','strike'] , how='left')

    return ce.ffill().bfill() , pe.ffill().bfill() 



In [164]:
def lower_bound(arr, target):
    low , high = 0, len(arr)
    while low < high:
        mid = low + (high -low)//2
        if arr['strike'].iloc[mid] < target:
            low = mid + 1
        else:
            high = mid
    return low


In [165]:
def generate_trades(ce,pe):

    # Define the structures
    put = {'pos': 0, 'signal_time': None, 'long_ema': None, 'short_ema': None, 'entry_time': None, 'strike': None,
            'entry_price': None, 'exit_time': None, 'exit_price': None , 'pnl': 0 ,'type':None, 'pnl_sum':0 , 'price':None}
    call = {'pos': 0, 'signal_time': None, 'long_ema': None, 'short_ema': None, 'entry_time': None, 'strike': None,
            'entry_price': None, 'exit_time': None, 'exit_price': None , 'pnl': 0 ,'type':None, 'pnl_sum':0 , 'price':None}

    # Initialize trades list
    trades = []

    for i in range(len(futures) - 1):
        row = futures.iloc[i]
        signal = row['Signal']
        price_selector = 1
        price = futures.iloc[i + 1]['close'] if price_selector == 1 else futures.iloc[i + 1]['open']
        dt = futures.iloc[i + 1]['date_timestamp']

        if signal == 1:
            if call['pos'] == 0:
                strikes = ce[ce['date_timestamp'] == dt][['date_timestamp', 'open', 'strike']]
                if not strikes.empty:
                    strikes.sort_values(by='strike', inplace=True)
                    var = price
                    strikes.reset_index(drop=True, inplace=True)
                    index = lower_bound(strikes, var)
                    if index >= len(strikes):
                        index -= 1
                    if index > 0:
                        index -= 1
                    buy_val = strikes['open'].iloc[index]
                    call.update({'pos': 1, 'price': row['close'], 'signal_time': row['date_timestamp'], 'long_ema': row['Long_EMA'],
                                'short_ema': row['Short_EMA'], 'strike': strikes['strike'].iloc[index],
                                'entry_time': dt, 'entry_price': buy_val})
                else:
                    print(f'No strikes on call option at {i} and {dt}')
            if put['pos'] == 1:
                strikes = pe[(pe['date_timestamp'] == dt) & (pe['strike'] == put['strike'])]
                if not strikes.empty:
                    var = strikes['open'].iloc[0]
                    put.update({'pos': 0, 'exit_time': dt, 'exit_price': var, 'pnl': var - put['entry_price'], 'type': 'PE'})
                    trades.append(put.copy())
                else:
                    print(f'No closing on put option at {i}, {dt}, {put["strike"]}')

        elif signal == -1:
            if put['pos'] == 0:
                strikes = pe[pe['date_timestamp'] == dt][['date_timestamp', 'open', 'strike']]
                if not strikes.empty:
                    strikes.sort_values(by='strike', inplace=True)
                    var = price
                    strikes.reset_index(drop=True, inplace=True)
                    index = lower_bound(strikes, var)
                    if index >= len(strikes):
                        index -= 1
                    if index < len(strikes):
                        index += 1
                    sell_val = strikes['open'].iloc[index]
                    put.update({'pos': 1, 'price': row['close'], 'signal_time': row['date_timestamp'], 'long_ema': row['Long_EMA'],
                                'short_ema': row['Short_EMA'], 'strike': strikes['strike'].iloc[index],
                                'entry_time': dt, 'entry_price': sell_val})
                else:
                    print(f'No strikes on put option at {i} and {dt}')
            if call['pos'] == 1:
                strikes = ce[(ce['date_timestamp'] == dt) & (ce['strike'] == call['strike'])]
                if not strikes.empty:
                    var = strikes['open'].iloc[0]
                    call.update({'pos': 0, 'exit_time': dt, 'exit_price': var, 'pnl': var - call['entry_price'], 'type': 'CE'})
                    trades.append(call.copy())
                else:
                    print(f'No closing on call option at {i}, {dt}, {call["strike"]}')

    # Final closing for any open positions at the end of the dataset
    if call['pos'] == 1:
        call_short = ce[(ce['date_timestamp'] == futures.iloc[-1]['date_timestamp']) & (ce['strike'] == call['strike'])]
        if not call_short.empty:
            var = call_short.iloc[0]['close']
            call.update({'pos': 0, 'exit_time': futures.iloc[-1]['date_timestamp'], 'exit_price': var, 'pnl': var - call['entry_price']})
            trades.append(call.copy())
        else:
            print('No closing on call option')

    if put['pos'] == 1:
        put_short = pe[(pe['date_timestamp'] == futures.iloc[-1]['date_timestamp']) & (pe['strike'] == put['strike'])]
        if not put_short.empty:
            var = put_short.iloc[0]['close']
            put.update({'pos': 0, 'exit_time': futures.iloc[-1]['date_timestamp'], 'exit_price': var, 'pnl': var - put['entry_price']})
            trades.append(put.copy())
        else:
            print('No closing on put option')

In [166]:
print(put['pos'],call['pos'])

0 0


In [167]:
for i in range(1,len(trades)):
    trades[i]['pnl_sum'] = trades[i-1]['pnl_sum'] + trades[i]['pnl']

In [168]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[i for i in range(len(trades))],
    y=[i['pnl'] for i in trades],
    mode='lines+markers',
    name='PnL'
))

fig.show()

In [169]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[i for i in range(len(trades))],
    y=[i['pnl_sum'] for i in trades],
    mode='lines+markers',
    name='PnL'
))

fig.show()

In [170]:
def drawdown(arr):
    max_val = -10000000000
    max_index , i = 0 , 0
    drawdown_val = -10000000000
    while i < len(arr):
        if arr[i]['pnl_sum'] > max_val:
            max_val = arr[i]['pnl_sum']
            while i < len(arr) and arr[i]['pnl_sum']<=max_val:
                drawdown_val = max(drawdown_val, max_val - arr[i]['pnl_sum'])
                i += 1

    return drawdown_val


In [171]:
drawdownval = drawdown(trades)
print(drawdownval)

197565.0


In [172]:
for i in trades:
    print(f" signal time {i['signal_time']} | price {i['price']} |  on date_time {i['entry_time']} | strike {i['strike']} | type : {i['type']}|  buy price {i['entry_price']}  |   sold at {i['exit_price']} on {i['exit_time']} , short_ema {i['short_ema']} | long_ema {i['long_ema']} |  PnL = {i['pnl']}")

 signal time 2023-12-01 09:16:00 | price 4482285 |  on date_time 2023-12-01 09:17:00 | strike 4500000 | type : PE|  buy price 67500.0  |   sold at 67000.0 on 2023-12-01 09:18:00 , short_ema 4482325.0 | long_ema 4482331.296296297 |  PnL = -500.0
 signal time 2023-12-01 09:17:00 | price 4483200 |  on date_time 2023-12-01 09:18:00 | strike 4480000 | type : CE|  buy price 62300.0  |   sold at 79750.0 on 2023-12-01 10:38:00 , short_ema 4482500.0 | long_ema 4482395.644718793 |  PnL = 17450.0
 signal time 2023-12-01 10:37:00 | price 4511600 |  on date_time 2023-12-01 10:38:00 | strike 4530000 | type : PE|  buy price 2955.0  |   sold at 67560.0 on 2023-12-01 10:39:00 , short_ema 4512317.106309336 | long_ema 4512325.156835257 |  PnL = 64605.0
 signal time 2023-12-01 10:38:00 | price 4512700 |  on date_time 2023-12-01 10:39:00 | strike 4510000 | type : CE|  buy price 62850.0  |   sold at 61080.0 on 2023-12-01 10:43:00 , short_ema 4512393.685047468 | long_ema 4512352.922995608 |  PnL = -1770.0
 s

In [173]:
pnl_per_day = {}

for trade in trades:
    if trade['entry_time'] not in pnl_per_day:
        pnl_per_day[trade['entry_time']] = []
    pnl_per_day[trade['entry_time']].append(-trade['entry_price'])

    if trade['exit_time'] not in pnl_per_day:
        pnl_per_day[trade['exit_time']] = []
    pnl_per_day[trade['exit_time']].append(trade['exit_price'])


net_pnl_per_day = {}

for day in pnl_per_day:

    net_pnl_per_day[day] = sum(pnl_per_day[day])
    

In [174]:
entry_per_day = {}
exit_per_day = {}

for trade in trades:
    if trade['entry_time'] not in entry_per_day:
        entry_per_day[trade['entry_time']] = 0.0
    entry_per_day[trade['entry_time']] += trade['entry_price']

    if trade['exit_time'] not in exit_per_day:
        exit_per_day[trade['exit_time']] = 0.0
    exit_per_day[trade['exit_time']] += trade['exit_price']


In [175]:
percentage_pnl = {}

for day in entry_per_day:
    percentage_pnl[day] = (net_pnl_per_day[day] / entry_per_day[day]) * 100

In [176]:
arr = np.array(list(percentage_pnl.values()))

mean = np.mean(arr)

std_dev = np.std(arr)

risk_free_rate = 0.0

sharpe_ratio = (mean - risk_free_rate) / std_dev

print(sharpe_ratio)

0.10466410139763432
